In [ ]:
#!/usr/bin/env python
import snowflake.connector as sfc
import pandas as pd
from datetime import datetime
import numpy as np
import ast
import re
import time
from snowflake.connector.pandas_tools import write_pandas
import os
from io import StringIO
from smart_open import smart_open

# Connect to DB

In [ ]:
omni = sfc.connect(
    user='cs2021_user3',
    password='Cap2021Stone-3',
    account='paciolan',
    warehouse='INTERNAL_ANALYTICS_WH',#'capstone2021_wh',
    database='omni_raw',
    schema='FUND',
    role = 'CAPSTONE2021_ROLE'
)

# Extract data

## 1. Account

### Get data from snowflake

In [ ]:
cs = omni.cursor()
# cur = cs.execute('''SELECT DISTINCT DATA:z_lastupdatedusertimestamp::string as z_lastupdatedusertimestamp from accounts''')
cur = cs.execute('''SELECT DISTINCT 
                    DATA:z_lastupdatedusertimestamp::string as z_lastupdatedusertimestamp,
                    DATA:accountname::string as account_name,
                    DATA:id::string as account_id,
                    DATA:dbid::string as client_id,
                    DATA:keywords as keywords, 
                    DATA:userdefinedfields as userdefinedfields,
                    DATA:tags as tags,
                    DATA:preferredemail::string as preferredemail,
                    DATA:preferredcontactmethod as preferredcontactmethod
                    from accounts  
                    limit 31728370
                    offset 31728370''')
print("execution_finished")
account_df2 = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
print(account_df2)


In [ ]:
account_df2.to_csv("all_accounts_2.csv")

In [ ]:
account_df2

### Connect to S3 

In [ ]:
import boto3
from botocore.client import Config
import s3fs 
import os
mykey = "here is our key, we delete it for submission"
mysecretkey = "key"
config = Config(connect_timeout=120, 
                retries={'max_attempts': 10})
s3 = boto3.resource(
    service_name='s3',
    region_name='us-west-2',
    aws_access_key_id=mykey,
    aws_secret_access_key=mysecretkey
)
client = boto3.client(
    service_name='s3',
    region_name='us-west-2',
    aws_access_key_id=mykey,
    aws_secret_access_key=mysecretkey,
    config=config
)
os.environ["AWS_DEFAULT_REGION"] = 'us-west-2'
os.environ["AWS_ACCESS_KEY_ID"] = mykey
os.environ["AWS_SECRET_ACCESS_KEY"] = mysecretkey

bucket = 'uci-capstone-custfields'


### Upload file to S3

In [ ]:
# import boto3
# s32 = boto3.resource('s3')
s3 = boto3.client('s3')
with open('all_accounts.csv', 'rb') as data:
    s3.upload_fileobj(data,
                      'uci-capstone-custfields',
                      'raw/all_accounts.csv')

In [ ]:
# import boto3
# s32 = boto3.resource('s3')
s3 = boto3.client('s3')
with open('all_accounts_2.csv', 'rb') as data:
    s3.upload_fileobj(data,
                      'uci-capstone-custfields',
                      'raw/all_accounts_2.csv')

### Clean Data First Round

In [ ]:
def convert_datetime(timestamp):
    if type(timestamp) == float and np.isnan(timestamp):
        return pd.to_datetime(timestamp)
    elif type(timestamp) == str and "T" in timestamp:
        return str(pd.to_datetime(timestamp)).replace("+00:00", "").split(".")[0]
    elif type(timestamp) == str and len(timestamp) == 0:
        return pd.to_datetime(np.nan)
    else:
        timestamp = int(timestamp)
        if np.isnan(timestamp):
            return pd.to_datetime(timestamp)
        return str(datetime.fromtimestamp(timestamp/1000.0)).split(".")[0]

In [ ]:
def to_dict(x):
    result = dict()
    if type(x) == str and x != "":
        x = ast.literal_eval("["+x+"]")
        for d in x:
            result[d["value1"]] = d["value2"]
        return str(result)[1:-1]
    return np.nan
            
def to_nan(x):
    if type(x) == str and x =="":
        return np.nan
    else:
        return x
    
def only_comma(x):
    return len(x.replace(",",""))==0
def remove_duplicates(x):
    if type(x) == str and x != "" and not only_comma(x):
        x = "{"+x+"}"
        x = ast.literal_eval(x)
        return str(x)
    return np.nan

def remove_duplicates2(x):
    if type(x) == str and x != "" and not only_comma(x):
        x = x.strip(",")
        x= re.sub(r',+',',',x)
        x = re.sub("[\{\}]","",x)
        x = "{"+x+"}"
        x = ast.literal_eval(x)
        return str(x)
    return np.nan       

In [ ]:
cols = ["keywords","userdefinedfields","tags"]
testing_orgid = ["SPORTS","ARTSDEMO","CLASS1","CLASS3A","CICD80"]
concat_cols = ["keywords","userdefinedfields","tags",
 "preferredemail","preferredcontactmethod","account_name"]
account_df2 = pd.read_csv("all_accounts.csv", chunksize = 100000)
for i,chunk in enumerate(account_df2):
    if i >= 281:
        print(i)
        print("before removing:",len(chunk))
        chunk.drop(columns = ["Unnamed: 0"], inplace =True)
        chunk.rename(columns = {"CLIENT_ID":"org_id","ACCOUNT_ID":"ACCOUNT_ID".lower(),
                               "ACCOUNT_NAME":"account_name","KEYWORDS":"KEYWORDS".lower(),
                               "USERDEFINEDFIELDS":"USERDEFINEDFIELDS".lower(),
                               "TAGS":"tags","PREFERREDEMAIL":"preferredemail",
                               "PREFERREDCONTACTMETHOD":"preferredcontactmethod"}, 
                     inplace = True)
        chunk["account_name"] = "\""+chunk["account_name"].apply(lambda x: x.replace("\"","\'").replace("\\","") if type(x)==str else x)+"\""
#         chunk["account_name"] = chunk["account_name"].apply(lambda x: x.replace("\\","") if type(x)==str else x)
        chunk["preferredemail"] ="\""+chunk["preferredemail"].apply(lambda x: x.replace("\"","\'").replace("\\","") if type(x)==str else x)+"\""
        chunk = chunk[~chunk.org_id.isin(testing_orgid)].copy()
        print("after removing testing accounts:",len(chunk))

        ### Simple Formatting
        for col in cols:
            chunk[col] = chunk[col].astype(str)
            chunk[col] = chunk[col].apply(lambda x: x.replace("\n ",""))
            chunk[col] = chunk[col].apply(lambda x: x.replace("[",""))
            chunk[col] = chunk[col].apply(lambda x: x.replace("]",""))
            chunk[col] = chunk[col].apply(lambda x: "" if x == "nan" else x)
            chunk[col] = chunk[col].apply(lambda x: x.replace("\n",""))
        chunk["Z_LASTUPDATEDUSERTIMESTAMP"] =chunk.Z_LASTUPDATEDUSERTIMESTAMP.apply(convert_datetime)
        chunk = chunk[pd.to_datetime(chunk.Z_LASTUPDATEDUSERTIMESTAMP)>=datetime(2018,1,1)].copy()
        chunk.sort_values(by = ["Z_LASTUPDATEDUSERTIMESTAMP"], inplace = True)
        result = chunk[["account_id", "org_id"]].copy()

        ### Merge into Set or Dict (only keep the earliest records)
        for col in concat_cols:
            print(col)
            if col == "userdefinedfields":
                chunk[col]=chunk[col].apply(to_dict)
            else:
                chunk[col] = chunk[col].apply(to_nan)
            temp = chunk[[col,"account_id",
                        "org_id"]].dropna()
            temp = pd.DataFrame(temp.groupby(["account_id",          
                               "org_id"])[col].apply(','.join).apply(str.strip)).reset_index()
            temp[col] = temp[col].apply(remove_duplicates)
            result = pd.merge(result,temp, on =["account_id", "org_id"], how = "outer")
            result.drop_duplicates(inplace = True)
        result.drop_duplicates(inplace = True)
        if i == 0:
            result.to_csv("s3://uci-capstone-custfields/raw/cleaned_accounts_v1.csv",
                          index = False)
        else:
            result.to_csv("s3://uci-capstone-custfields/raw/cleaned_accounts_v1.csv", 
                        mode = "a", header = False, index = False)
        print(len(result))


In [ ]:
# cols = ["keywords","userdefinedfields","tags"]
# testing_orgid = ["SPORTS","ARTSDEMO","CLASS1","CLASS3A","CICD80"]
# concat_cols = ["keywords","userdefinedfields","tags",
#  "preferredemail","preferredcontactmethod","account_name"]
# chunk = pd.read_csv("all_accounts_2.csv")
# print("before removing:",len(chunk))
# chunk.drop(columns = ["Unnamed: 0"], inplace =True)
# chunk.rename(columns = {"CLIENT_ID":"org_id","ACCOUNT_ID":"ACCOUNT_ID".lower(),
#                        "ACCOUNT_NAME":"account_name","KEYWORDS":"KEYWORDS".lower(),
#                        "USERDEFINEDFIELDS":"USERDEFINEDFIELDS".lower(),
#                        "TAGS":"tags","PREFERREDEMAIL":"preferredemail",
#                        "PREFERREDCONTACTMETHOD":"preferredcontactmethod"}, 
#              inplace = True)
# chunk["account_name"] = "\""+chunk["account_name"].apply(lambda x: x.replace("\"","\'").replace("\\","") if type(x)==str else x)+"\""
# #         chunk["account_name"] = chunk["account_name"].apply(lambda x: x.replace("\\","") if type(x)==str else x)
# chunk["preferredemail"] ="\""+chunk["preferredemail"].apply(lambda x: x.replace("\"","\'").replace("\\","") if type(x)==str else x)+"\""
# chunk = chunk[~chunk.org_id.isin(testing_orgid)].copy()
# print("after removing testing accounts:",len(chunk))

# ### Simple Formatting
# for col in cols:
#     chunk[col] = chunk[col].astype(str)
#     chunk[col] = chunk[col].apply(lambda x: x.replace("\n ",""))
#     chunk[col] = chunk[col].apply(lambda x: x.replace("[",""))
#     chunk[col] = chunk[col].apply(lambda x: x.replace("]",""))
#     chunk[col] = chunk[col].apply(lambda x: "" if x == "nan" else x)
#     chunk[col] = chunk[col].apply(lambda x: x.replace("\n",""))
chunk["Z_LASTUPDATEDUSERTIMESTAMP"] =chunk.Z_LASTUPDATEDUSERTIMESTAMP.apply(convert_datetime)
chunk = chunk[pd.to_datetime(chunk.Z_LASTUPDATEDUSERTIMESTAMP)>=datetime(2018,1,1)].copy()
chunk.sort_values(by = ["Z_LASTUPDATEDUSERTIMESTAMP"], inplace = True)
result = chunk[["account_id", "org_id"]].copy()

### Merge into Set or Dict (only keep the earliest records)
for col in concat_cols:
    print(col)
    if col == "userdefinedfields":
        chunk[col]=chunk[col].apply(to_dict)
    else:
        chunk[col] = chunk[col].apply(to_nan)
    temp = chunk[[col,"account_id",
                "org_id"]].dropna()
    temp = pd.DataFrame(temp.groupby(["account_id",          
                       "org_id"])[col].apply(','.join).apply(str.strip)).reset_index()
    temp[col] = temp[col].apply(remove_duplicates)
    result = pd.merge(result,temp, on =["account_id", "org_id"], how = "outer")
    result.drop_duplicates(inplace = True)
result.drop_duplicates(inplace = True)
result.to_csv("s3://uci-capstone-custfields/raw/cleaned_accounts_2_v1.csv",
                  index = False)
result.to_csv("cleaned_accounts_2_v1.csv",
                  index = False)
print(len(result))


In [ ]:
result

### Clean Second Round

In [ ]:
cols = ["keywords","userdefinedfields","tags"]
concat_cols = ["keywords","userdefinedfields","tags",
 "preferredemail","preferredcontactmethod","account_name"]
columns = ["account_id","org_id","keywords","userdefinedfields",
           "tags","preferredemail","preferredcontactmethod",
           "account_name"]
# account_df2 = pd.read_csv("cleaned_accounts_v1.csv", chunksize = 100000)
for k in range(3,4):
    
    account_df2 = pd.read_csv(f"cleaned_v1/cleaned_accounts_v1 ({k}).csv", 
                              chunksize = 100000,
                              names = columns)
    for i,chunk in enumerate(account_df2):   
        if i >= 0:
            print(i)
            print("before removing:",len(chunk))
            if len(chunk) != len(chunk.drop_duplicates(subset = ["account_id","org_id"])):

                ### Simple Formatting
                for col in concat_cols:
                    chunk[col] = chunk[col].apply(lambda x: x[1:-1] if type(x)==str and "{" in x else x)

                ### Merge into Set or Dict (only keep the earliest records)
                result = chunk.fillna("").groupby(["account_id","org_id"], 
                                      as_index = False).agg({"keywords":",".join,
                                                            "userdefinedfields":",".join,
                                                            "tags":",".join,
                                                            "preferredemail":",".join,
                                                            "preferredcontactmethod":",".join,
                                                            "account_name":",".join}) 
                for col in concat_cols:
                    result[col] = result[col].apply(remove_duplicates2)

#                 if i == 0:
                result.to_csv(f"cleaned_v2/cleaned_accounts_v2_{k}_{i}.csv", index = False)
#                 else:
#                     result.to_csv("cleaned_v2/cleaned_accounts_v2_{k}_{i}.csv", 
#                                 mode = "a", header = False, index = False)
                print(len(result))
            else:
#                 if i == 0:
                chunk.to_csv(f"cleaned_v2/cleaned_accounts_v2_{k}_{i}.csv", index = False)
#                 else:
#                     chunk.to_csv("cleaned_v2/cleaned_accounts_v2_{k}_{i}.csv", 
#                                 mode = "a", header = False, index = False)




### Clean Third Round

In [ ]:
cols = ["keywords","userdefinedfields","tags"]
concat_cols = ["keywords","userdefinedfields","tags",
 "preferredemail","preferredcontactmethod","account_name"]
columns = ["account_id","org_id","keywords","userdefinedfields",
           "tags","preferredemail","preferredcontactmethod",
           "account_name"]
# account_df2 = pd.read_csv("cleaned_accounts_v1.csv", chunksize = 100000)
######
# chunk = pd.read_csv("cleaned_v2/cleaned_accounts_v2_{i}.csv",
#                     names = columns)
######
# chunk = []
# for i in range(0,94):
#     print(i)
#     chunk.append(pd.read_csv(f"cleaned_v2/cleaned_accounts_v2_1_{i}.csv"))
######
# chunk = []
# for i in range(0,72):
#     print(i)
#     chunk.append(pd.read_csv(f"cleaned_v2/cleaned_accounts_v2_2_{i}.csv"))
######
chunk = []
for i in range(0,27):
    print(i)
    chunk.append(pd.read_csv(f"cleaned_v2/cleaned_accounts_v2_3_{i}.csv"))
chunk = pd.concat(chunk)
if len(chunk) != len(chunk.drop_duplicates(subset = ["account_id","org_id"])):
    print("len(chunk):",len(chunk))
    

    ### Simple Formatting
    for col in concat_cols:
        print("col1:",col)
        chunk[col] = chunk[col].apply(lambda x: x[1:-1] if type(x)==str and "{" in x else x)

    ### Merge into Set or Dict (only keep the earliest records)
    result = chunk.fillna("").groupby(["account_id","org_id"], 
                          as_index = False).agg({"keywords":",".join,
                                                "userdefinedfields":",".join,
                                                "tags":",".join,
                                                "preferredemail":",".join,
                                                "preferredcontactmethod":",".join,
                                                "account_name":",".join}) 
    for col in concat_cols:
        print("col2:",col)
        result[col] = result[col].apply(remove_duplicates2)
    result.to_csv(f"cleaned_v3/cleaned_accounts_v3_3.csv", index = False)

    print(len(result))
else:
    chunk.to_csv(f"cleaned_v3/cleaned_accounts_v3_3.csv", index = False)





### Clean Fourth Round

In [ ]:
cols = ["keywords","userdefinedfields","tags"]
concat_cols = ["keywords","userdefinedfields","tags",
 "preferredemail","preferredcontactmethod","account_name"]
columns = ["account_id","org_id","keywords","userdefinedfields",
           "tags","preferredemail","preferredcontactmethod",
           "account_name"]
chunk = []
for i in range(0,4):
    print(i)
    chunk.append(pd.read_csv(f"cleaned_v3/cleaned_accounts_v3_{i}.csv"))
chunk = pd.concat(chunk)
if len(chunk) != len(chunk.drop_duplicates(subset = ["account_id","org_id"])):
    print("len(chunk):",len(chunk))
    
    ### Simple Formatting
    for col in concat_cols:
        print("col1:",col)
        chunk[col] = chunk[col].apply(lambda x: x[1:-1] if type(x)==str and "{" in x else x)

    ### Merge into Set or Dict (only keep the earliest records)
    result = chunk.fillna("").groupby(["account_id","org_id"], 
                          as_index = False).agg({"keywords":",".join,
                                                "userdefinedfields":",".join,
                                                "tags":",".join,
                                                "preferredemail":",".join,
                                                "preferredcontactmethod":",".join,
                                                "account_name":",".join}) 
    for col in concat_cols:
        print("col2:",col)
        result[col] = result[col].apply(remove_duplicates2)
    result.to_csv(f"cleaned_v4/cleaned_accounts_v4.csv", index = False)

    print(len(result))
else:
    chunk.to_csv(f"cleaned_v4/cleaned_accounts_v4.csv", index = False)





In [ ]:
result.to_csv(f"cleaned_v4/cleaned_accounts_v4.csv", index = False)

In [ ]:
print(len(result))

### Upload Cleaned data to s3

In [ ]:
s3 = boto3.client('s3')
with open('cleaned_v4/cleaned_accounts_v4.csv', 'rb') as data:
    s3.upload_fileobj(data,
                      'uci-capstone-custfields',
                      'raw/cleaned_accounts_v4.csv')

In [ ]:
for data in pd.read_csv("cleaned_v4/cleaned_accounts_v4.csv", chunksize = 1000):
    print(data)
    break

In [ ]:
data

### Adding Missing Ones 

In [ ]:
# first = pd.read_csv("cleaned_v4/cleaned_accounts_v4.csv")
second = pd.read_csv("cleaned_accounts_2_v1.csv")

In [ ]:
chunk = pd.concat([first,second])

In [ ]:
cols = ["keywords","userdefinedfields","tags"]
concat_cols = ["keywords","userdefinedfields","tags",
 "preferredemail","preferredcontactmethod","account_name"]
columns = ["account_id","org_id","keywords","userdefinedfields",
           "tags","preferredemail","preferredcontactmethod",
           "account_name"]

if len(chunk) != len(chunk.drop_duplicates(subset = ["account_id","org_id"])):
    print("len(chunk):",len(chunk))
    
    ### Simple Formatting
    for col in concat_cols:
        print("col1:",col)
        chunk[col] = chunk[col].apply(lambda x: x[1:-1] if type(x)==str and "{" in x else x)

    ### Merge into Set or Dict (only keep the earliest records)
    result = chunk.fillna("").groupby(["account_id","org_id"], 
                          as_index = False).agg({"keywords":",".join,
                                                "userdefinedfields":",".join,
                                                "tags":",".join,
                                                "preferredemail":",".join,
                                                "preferredcontactmethod":",".join,
                                                "account_name":",".join}) 
    for col in concat_cols:
        print("col2:",col)
        result[col] = result[col].apply(remove_duplicates2)
    result.to_csv(f"cleaned_v5/cleaned_accounts_v5.csv", index = False)

    print(len(result))
else:
    chunk.to_csv(f"cleaned_v5/cleaned_accounts_v5.csv", index = False)


In [ ]:
for col in concat_cols:
    print("col2:",col)
    result[col] = result[col].apply(remove_duplicates2)
result.to_csv(f"cleaned_v5/cleaned_accounts_v5.csv", index = False)
result

In [ ]:
s3 = boto3.client('s3')
with open('cleaned_v5/cleaned_accounts_v5.csv', 'rb') as data:
    s3.upload_fileobj(data,
                      'uci-capstone-custfields',
                      'raw/cleaned_accounts_v5.csv')

In [ ]:
cs = omni.cursor()
cur = cs.execute('''SELECT DISTINCT DATA:orgid::string AS org_id,
                    DATA:id::string as account_id,
                    DATA:dbid::string as client_id,
                    DATA:categories as categories,
                    DATA:currentpoints as currentpoints,
                    DATA:donormemberships AS donormemberships,
                    DATA:firstallociddriveyears AS firstallociddriveyears,
                    DATA:firstdate AS firstdate,
                    DATA:firstgiftamount AS firstgiftamount,
                    DATA:greatestallociddriveyears AS greatestallociddriveyears,
                    DATA:greatestdate AS greatestdate,
                    DATA:greatestgiftamount AS greatestgiftamount,
                    DATA:lastallociddriveyears AS lastallociddriveyears,
                    DATA:lastdate AS lastdate,
                    DATA:lastgiftamount AS lastgiftamount,
                    DATA:lastupdatemembership AS lastupdatemembership,
                    DATA:lastupdateprioritypoints AS lastupdateprioritypoints,
                    DATA:lastupdatestatistics AS lastupdatestatistics,
                    DATA:lifetimedonationamount AS lifetimedonationamount,
                    DATA:pointsrank::integer AS pointsrank,
                    DATA:yearsofdonating::integer AS yearsofdonating
                    from donors where lastdate>20180101
                    limit 10''')
# WHERE LEN(DATA:keywords::STRING) > 0
# for col1 in cur:
# #     df = {"account_name":col1[0],
# #          }
# #     acct_list.append()
#     print(col1)

donor_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
print(donor_df)

## 2. Donor Categories

In [ ]:
#cleaned donor cat
cs = omni.cursor()
cur = cs.execute('''SELECT DISTINCT DATA:id::string as donor_cat_id,
                    DATA:dbid::string as org_id,
                    DATA:name:en_US::string as donor_cat_name
                    from donorcategories 
                    where org_id not in ('SPORTS','ARTSDEMO','CLASS1','CLASS3A','CICD80')
                    ''')
# WHERE LEN(DATA:keywords::STRING) > 0
# for col1 in cur:
# #     df = {"account_name":col1[0],
# #          }
# #     acct_list.append()
#     print(col1)

donor_cat_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
donor_cat_df=donor_cat_df.rename(columns=str.lower)
donor_cat_df

In [ ]:
#3328rows*3cols 
donor_cat_df.to_csv("s3://uci-capstone-custfields/cleaned_donor_category.csv", 
                             index = False)

## 3. Organizations

In [ ]:
#cleaned org
cs = omni.cursor()
cur = cs.execute('''SELECT DISTINCT DATA:id::string as org_id,
                    DATA:description:en_US::string as description,
                    DATA:name:en_US::string AS name
                    from organizations
                    where org_id not in ('SPORTS','ARTSDEMO','CLASS1','CLASS3A','CICD80')
                    ''')
# WHERE LEN(DATA:keywords::STRING) > 0
# for col1 in cur:
# #     df = {"account_name":col1[0],
# #          }
# #     acct_list.append()
#     print(col1)

organization_df = pd.DataFrame.from_records(iter(cur), 
                                            columns=[x[0] for x in cur.description])
organization_df=organization_df.rename(columns=str.lower)
organization_df

In [ ]:
#293rows*3cols
#exclude accountdbid
organization_df.to_csv("s3://uci-capstone-custfields/cleaned_organizations.csv", 
                             index = False)

## 4. Allocations

In [ ]:
#clean allo
cs = omni.cursor()
cur = cs.execute('''SELECT DISTINCT 
                    DATA:dbid::string AS org_id,
                    DATA:allocationgroupid::string as allocationgroupid,
                    DATA:id::string AS alloc_id,
                    DATA:annualgift::boolean as annualgift,
                    DATA:consumerfunddescription::string AS consumerfunddescription,
                    DATA:charitablepercent as charitablepercent,
                    DATA:lifetimegiving::boolean AS lifetimegiving,
                    DATA:name:en_US::string AS name,
                    DATA:prioritypointpercent AS prioritypointpercent,
                    DATA:deadlineday AS deadlineday,
                    DATA:deadlinefactor AS deadlinefactor,
                    DATA:deadlinemonth AS deadlinemonth,
                    DATA:deadlineyear AS deadlineyear,
                    DATA:isexcludedfromdonationhistory AS isexcludedfromdonationhistory,
                    DATA:yearsofgiving AS yearsofgiving
                    from allocations
                    where org_id not in ('SPORTS','ARTSDEMO','CLASS1','CLASS3A','CICD80')
                    ''')


# WHERE LEN(DATA:keywords::STRING) > 0
# for col1 in cur:
# #     df = {"account_name":col1[0],
# #          }
# #     acct_list.append()
#     print(col1)

alloc_df = pd.DataFrame.from_records(iter(cur), 
                                            columns=[x[0] for x in cur.description])
alloc_df=alloc_df.rename(columns=str.lower)
alloc_df

In [ ]:
#10849rows*15cols
alloc_df.to_csv("s3://uci-capstone-custfields/cleaned_allocations.csv", 
                             index = False)

## 5. Alloc group

In [ ]:
#cleaned alloc group
cs = omni.cursor()
cur = cs.execute('''SELECT DISTINCT 
                    DATA:dbid::string AS org_id,
                    DATA:id::string AS allocationgroupid,
                    DATA:name:en_US::string AS name
                    from allocgroups
                    where org_id not in ('SPORTS','ARTSDEMO','CLASS1','CLASS3A','CICD80')
                    ''')
# WHERE LEN(DATA:keywords::STRING) > 0
# for col1 in cur:
# #     df = {"account_name":col1[0],
# #          }
# #     acct_list.append()
#     print(col1)

alloc_grp_df = pd.DataFrame.from_records(iter(cur), 
                                            columns=[x[0] for x in cur.description])
alloc_grp_df=alloc_grp_df.rename(columns=str.lower)
alloc_grp_df


In [ ]:
#216rows*3col
alloc_grp_df.to_csv("s3://uci-capstone-custfields/cleaned_allocations_group.csv", 
                             index = False)

## 6. Transaction Items

In [ ]:
#cleaned trans
cs = omni.cursor()
cur = cs.execute('''SELECT  
                    DATA:dbid::string AS org_id,
                    DATA:accountid::string AS account_id,
                    DATA:allocationid::string AS allocationid,
                    DATA:allocgroupid::string AS allocgroupid,
                    DATA:transactionid::string AS transactionid,
                    DATA:id::string AS transaction_item_id,
                    DATA:batchid::string AS batchid,
                    DATA:channelid::string AS channelid,
                    DATA:allocationid::string AS allocationid,
                    DATA:creditamount::string AS creditamount,
                    DATA:driveyear AS driveyear,
                    CONCAT(SUBSTRING(DATA:receiveddate::string,1,10),' ',SUBSTRING(DATA:receiveddate::string,12,8)) AS receiveddate,
                    DATA:fundtype::string AS fundtype,
                    DATA:giftid::string AS giftid,
                    DATA:pledgeamount::string AS pledgeamount,
                    DATA:paymentamount::string AS paymentamount,
                    DATA:matchpledgeamount::string AS matchpledgeamount,
                    DATA:paymentapplyamount::string AS paymentapplyamount,
                    DATA:motiveid::string AS motiveid
                    from transactionitems 
                    where org_id not in ('SPORTS','ARTSDEMO','CLASS1','CLASS3A','CICD80')
                    AND SUBSTRING(receiveddate,1,4) in ('2015','2016','2017','2018','2019','2020','2021')
                    ''')
# WHERE LEN(DATA:keywords::STRING) > 0
# for col1 in cur:
# #     df = {"account_name":col1[0],
# #          }
# #     acct_list.append()
#     print(col1)

transactionItem_df = pd.DataFrame.from_records(iter(cur), 
                                            columns=[x[0] for x in cur.description])
transactionItem_df=transactionItem_df.rename(columns=str.lower)
transactionItem_df


### Further manipulation

In [ ]:
transactionItem_df['paymentamount']= transactionItem_df['paymentamount'].astype(int)/100
transactionItem_df['year'] = transactionItem_df['receiveddate'].apply(lambda x: x[:4])
yearlyorgpayment = transactionItem_df[['org_id','account_id','paymentamount','year']].groupby(['org_id','account_id','year']).sum().reset_index()
yearlyorgpayment_perperson = yearlyorgpayment.groupby(['org_id','year']).agg(["sum","mean","median","min","max","var","size"]).reset_index()
yearlyorgpayment_perperson.columns = ["_".join(x) for x in yearlyorgpayment_perperson.columns.ravel()]


In [ ]:
yearlyorgpayment_perperson.rename(str.lower,axis='columns',inplace=True)
yearlyorgpayment_perperson.rename(columns={'org_id_':'org_id','year_':'year'},inplace=True)
yearlyorgpayment_perperson

In [ ]:
yearlyorgpayment_perperson.to_csv("s3://uci-capstone-custfields/yearlyorgpayment_perperson.csv", 
                             index = False)

## 7. DIM_DONOR

In [ ]:
# raw data from DIM_DONOR
cs = omni.cursor()
cur = cs.execute('''SELECT *
                    from dim_donor
                    where organization_id != 'SPORTS'
                    and organization_id != 'ARTSDEMO'
                    and organization_id != 'CLASS1'
                    and organization_id != 'CLASS3A'
                    and organization_id != 'CICD80'
                ''')

donor_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
csv_buffer = StringIO()
donor_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/dim_donors.csv').put(Body=csv_buffer.getvalue())

In [ ]:
# clean and reformat for future use
donor_df = donor_df.drop(['DIM_ACCOUNT_KEY','DONOR_TYPE','DONOR_STATUS','DONOR_MEMBERSHIPS','MEMBERSHIPS'],axis=1)
donor_df['DONOR_TYPE_NAME'] = donor_df['DONOR_TYPE_NAME'].str.strip('{\n }')
donor_df['DONOR_TYPE_NAME'] = donor_df['DONOR_TYPE_NAME'].str.replace('"en_US":','')
donor_df['DONOR_STATUS_NAME'] = donor_df['DONOR_STATUS_NAME'].str.strip('{\n }')
donor_df['DONOR_STATUS_NAME'] = donor_df['DONOR_STATUS_NAME'].str.replace('"en_US":','')
donor_df['CATEGORIES'] = donor_df['CATEGORIES'].str.strip('{\n }')
donor_df['CATEGORY_START_DATES'] = donor_df['CATEGORY_START_DATES'].str.replace("{}","None")
donor_df['CATEGORY_END_DATES'] = donor_df['CATEGORY_END_DATES'].str.replace("{}","None")

donor_df.rename(columns={'ORGANIZATION_ID':'org_id','ID':'account_id', 'IS_DELETED':'is_deleted',
                                    'DONOR_TYPE_NAME':'donor_type','DONOR_STATUS_NAME':'donor_status',
                                    'YEARS_OF_DONATING':'years_of_donating','CATEGORIES':'categories',
                                    'CATEGORY_START_DATES':'cat_start_dates','CATEGORY_END_DATES':'cat_end_dates',
                                    'SYS_STATUS':'sys_status'
                                    }, inplace=True)

In [ ]:
# save the cleaned version to s3
csv_buffer = StringIO()
donor_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/cleaned_dim_donors.csv').put(Body=csv_buffer.getvalue())

## 8. DIM_DONOR_MEMBERSHIP 

In [ ]:
# raw 
cs = omni.cursor()
cur = cs.execute('''SELECT *
                    from dim_donor_membership
                    where organization_id != 'SPORTS'
                    and organization_id != 'ARTSDEMO'
                    and organization_id != 'CLASS1'
                    and organization_id != 'CLASS3A'
                    and organization_id != 'CICD80'
                ''')

donor_membership_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
csv_buffer = StringIO()
donor_membership_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/dim_donor_membership.csv').put(Body=csv_buffer.getvalue())

In [ ]:
donor_membership_df = donor_membership_df.drop(['DIM_ACCOUNT_KEY','DONOR_MEMBERSHIPS'],axis=1)
donor_membership_df.rename(columns={'ORGANIZATION_ID':'org_id','ACCOUNT_ID':'account_id', 'MEMBERSHIP_ID':'membership_id',
                                    'MEMBERSHIP_DRIVE_YEAR':'membership_drive_year','PLEDGED_LEVEL_NAME':'pledged_level',
                                    'RECEIPT_LEVEL_NAME':'receipt_level','TOTAL_DONATED':'total_donated',
                                    'TOTAL_PLEDGED':'total_pledged','DONOR_SYS_STATUS':'sys_status'}, inplace=True)
donor_membership_df

In [ ]:
# save the cleaned version to s3
csv_buffer = StringIO()
donor_membership_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/cleaned_dim_donor_membership.csv').put(Body=csv_buffer.getvalue())

In [ ]:
year_donation = donor_df[['org_id','membership_drive_year','total_donated','total_pledged']].groupby(['org_id','membership_drive_year']).agg(['sum','median','mean','var','min','max']).reset_index()
year_donation.columns = ["_".join(x) if x[0] not in ["org_id","year"] else x[0] for x in year_donation.columns.ravel()]
yea_donation = year_donation.rename(columns={'membership_drive_year_':'year'},inplace = True)

In [ ]:
# save the cleaned version to s3
csv_buffer = StringIO()
year_donation.to_csv(csv_buffer,index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/cleaned_year_donation.csv').put(Body=csv_buffer.getvalue())

## 9. AccountPriorityPointsProgramDetails

In [ ]:
cs = omni.cursor()
cur = cs.execute('''select DISTINCT DATA:acctid::string as account_id,
                           DATA:actualpoints::string as actual_points,
                           DATA:dbid::string as org_id,
                           DATA:pointscategoriesbreakdown as points_catg,
                           DATA:potentialpoints::int as potential_points,
                           DATA:progid::string as prog_id,
                           DATA:seq::int as seq
                    from ACCOUNTPRIORITYPOINTSPROGRAMDETAILS''')

priority_prog_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
# save the raw version
csv_buffer = StringIO()
priority_prog_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/priority_program.csv').put(Body=csv_buffer.getvalue())

In [ ]:
priority_prog_df = pd.read_csv(smart_open('s3://uci-capstone-custfields/raw/priority_program.csv'))

In [ ]:
# clean json format
priority_prog_df['POINTS_CATG'] = priority_prog_df['POINTS_CATG'].str.replace('{\n    "actualpoints": ','')
priority_prog_df['POINTS_CATG'] = priority_prog_df['POINTS_CATG'].str.replace(',\n    "name": {\n      "en_US":',',')
priority_prog_df['POINTS_CATG'] = priority_prog_df['POINTS_CATG'].str.replace('\n    },\n    "potentialpoints": 0\n  },\n  ',',')
priority_prog_df['POINTS_CATG'] = priority_prog_df['POINTS_CATG'].str.replace('\n    },\n    "potentialpoints": 0\n  }\n','')
priority_prog_df['POINTS_CATG'] = priority_prog_df['POINTS_CATG'].str.replace('\n ','')

In [ ]:
priority_prog_df.rename(columns={'ACCOUNT_ID':'account_id','ACTUAL_POINTS':'actual_points',
                                'ORG_ID':'org_id','POINTS_CATG':'points_catg','POTENTIAL_POINTS':'potential_points',
                                'PROG_ID':'prog_id','SEQ':'seq'},inplace=True)
priority_prog_df = priority_prog_df.drop(['Unnamed: 0'],axis=1)
priority_prog_df

In [ ]:
#  save the cleaned version to s3
csv_buffer = StringIO()
priority_prog_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object('uci-capstone-custfields', 'raw/cleaned_priority_program.csv').put(Body=csv_buffer.getvalue())

## 10. Ticket -> events data [3 datasets]

In [ ]:
ticketing = sfc.connect(
    user='cs2021_user3',
    password='Cap2021Stone-3',
    account='paciolan',
    warehouse='capstone2021_wh',#'capstone2021_wh',
    database='omni_raw',
    schema='ticketing',
    role = 'CAPSTONE2021_ROLE'
)

### Ticket Season

In [ ]:
cs = ticketing.cursor()
cur = cs.execute('''SELECT DISTINCT DATA:ACTIVITY::string as ACTIVITY,
                    DATA:SOURCE_ID::string as ORG_ID,
                    DATA:LAST_DATETIME::datetime as LAST_DATETIME,
                    DATA:NAME::string AS NAME,
                    DATA:SEASON::string AS SEASON,
                    DATA:SIZE::string AS SIZE
                    from "CAPSTONE2020"."TICKETING"."TK_SEASON"''')

tk_season_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
tk_season_df.columns = [c.lower() for c in tk_season_df.columns]
print(len(tk_season_df))
tk_season_df.to_csv("ticket_season.csv", index = False)

In [ ]:
s3 = boto3.client('s3')
with open('ticket_season.csv', 'rb') as data:
    s3.upload_fileobj(data,
                      'uci-capstone-custfields',
                      'raw/ticket_season.csv')

In [ ]:
tk_season_df = pd.read_csv("ticket_season.csv")

In [ ]:
tk_season_df

### Ticket Region ZIP (VMC may be vendind machine control???  VMC is unique but zip is not)

In [ ]:
cs = ticketing.cursor()
cur = cs.execute('''SELECT DISTINCT 
                    DATA:SOURCE_ID::string as ORG_ID,
                    DATA:VMC::string as VMC,
                    DATA:ZIP::string AS ZIP
                    from "CAPSTONE2020"."TICKETING"."TK_REGION_ZIP"''')

tk_region_zip_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
tk_region_zip_df.columns = [c.lower() for c in tk_region_zip_df.columns]
print(len(tk_region_zip_df))
tk_region_zip_df.to_csv("ticket_region_zip.csv", index = False)

In [ ]:
s3 = boto3.client('s3')

with open('ticket_region_zip.csv', 'rb') as data:
    s3.upload_fileobj(data,
                      'uci-capstone-custfields',
                      'raw/ticket_region_zip.csv')

In [ ]:
tk_region_zip_df[tk_region_zip_df.org_id.str.contains("NCSU")]

### Ticket Transaction Item Event

In [ ]:
cs = ticketing.cursor()
start = time.time()
cur = cs.execute('''SELECT DATA:CUSTOMER::string as account_id,
                    DATA:SOURCE_ID::string as ORG_ID,
                    DATA:TRANS_NO::string as TRANS_NO,
                    DATA:VMC AS VMC,
                    DATA:DATE::datetime as DATE,
                    DATA:EVENT::string AS EVENT,
                    DATA:SEASON::STRING AS SEASON,
                    DATA:E_OQTY AS E_OQTY,
                    DATA:E_PL::string AS E_PL,
                    DATA:E_PRICE::string AS E_PRICE,
                    DATA:TOTAL_EPAY AS TOTAL_EPAY,
                    DATA:TOTAL_CPAY AS TOTAL_CPAY,
                    DATA:TOTAL_FPAY AS TOTAL_FPAY,
                    DATA:TOTAL_SPAY AS TOTAL_SPAY
                    from "CAPSTONE2020"."TICKETING"."TK_TRANS_ITEM_EVENT" 
                    where YEAR(DATE) >= 2018''')
end1 = time.time()
print("execute sql:",end1 - start)
tk_trans_item_event_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
end2 = time.time()
print("convert to pandas df:",end2 - end1)
tk_trans_item_event_df.columns = [c.lower() for c in tk_trans_item_event_df.columns]
print(len(tk_trans_item_event_df))
tk_trans_item_event_df.to_csv("ticket_trans_item_event.csv", index = False)
end3 = time.time()
print("save as csv:",end3 - end2)
print(tk_trans_item_event_df)

In [ ]:
tk_trans_item_event_df

In [ ]:
s3 = boto3.client('s3')

with open('ticket_trans_item_event.csv', 'rb') as data:
    s3.upload_fileobj(data,
                      'uci-capstone-custfields',
                      'raw/ticket_trans_item_event.csv')

In [ ]:
tk_trans_item_event_df= pd.read_csv("ticket_trans_item_event.csv")

In [ ]:
merge_df = pd.merge(tk_trans_item_event_df,tk_season_df, on= ["season","org_id"]).reset_index()

In [ ]:
merge_df["year"] = merge_df.date.apply(lambda x:x[:4])

In [ ]:
org_yearly_pay = merge_df[["org_id","account_id","size","year","total_epay","total_cpay",
          "total_fpay","total_spay"]].groupby(["org_id","size","year",
                                               "account_id"]).sum().reset_index()
org_yearly_pay["total_pay_all"] = org_yearly_pay.total_epay+org_yearly_pay.total_cpay+org_yearly_pay.total_fpay+org_yearly_pay.total_fpay

#### Version 1

In [ ]:
org_yearly_pperson = org_yearly_pay[["org_id","size","total_pay_all","total_epay",
                                     "year"]].groupby(["org_id","size",
                                             "year"]).agg(["sum","mean","median","min","max",
                                                           "var","size"]).reset_index()


In [ ]:
org_yearly_pperson.columns = ["_".join(x) if x[0] not in ["org_id","size","year"] else x[0] for x in org_yearly_pperson.columns.ravel()]

In [ ]:
org_yearly_pperson.to_csv("org_yearly_tk_pperson_v1.csv")

In [ ]:
org_yearly_pperson_pivot = org_yearly_pperson.pivot(index = ["org_id"], 
                                                    columns = ["size","year"],
                                                    values = [c for c in org_yearly_pperson.columns if c not in ["org_id","size","year"]])

In [ ]:
org_yearly_pperson_pivot.total_pay_all_sum

####  Version 2

In [ ]:
org_yearly_pay_v2 = org_yearly_pay.pivot(index = ["org_id", "account_id","year"], 
                     columns = ["size"],
                     values = ["total_pay_all","total_epay"]).reset_index()

In [ ]:
org_yearly_pay_v2.columns = ["_".join(x) if x[0] not in ["org_id","account_id","year"] else x[0] for x in org_yearly_pay_v2.columns.ravel()]

In [ ]:
large = ["L","H","T"]
small = ["S","M"]

In [ ]:
org_yearly_pay_v2["total_pay_all_large"] = org_yearly_pay_v2[["total_pay_all_T","total_pay_all_L",
                                                              "total_pay_all_H"]].sum(axis = 1)
org_yearly_pay_v2["total_pay_all_small"] = org_yearly_pay_v2[["total_pay_all_S",
                                                              "total_pay_all_M"]].sum(axis = 1)
org_yearly_pay_pperson_v2 = org_yearly_pay_v2[["org_id","year","total_pay_all_large",
                   "total_pay_all_small"]].groupby(["org_id","year"]).agg(["sum","mean","median",
                                                                           "min","max",
                                                           "var","size"]).reset_index()
org_yearly_pay_pperson_v2.columns = ["_".join(x) if x[0] not in ["org_id","year"] else x[0] for x in org_yearly_pay_pperson_v2.columns.ravel()]
org_yearly_pperson_v2_pivot = org_yearly_pay_pperson_v2.pivot(index = ["org_id"], 
                     columns = ["year"],
                     values = [c for c in org_yearly_pay_pperson_v2.columns if c not in ["org_id","year"]]).reset_index()
org_yearly_pperson_v2_pivot.columns = ["_".join(x) if x[0] !="org_id" else x[0] for x in org_yearly_pperson_v2_pivot.columns.ravel()]


In [ ]:
org_yearly_pperson_v2_pivot.to_csv("s3://uci-capstone-custfields/org_yearly_tk_pperson_v2.csv", 
                                 index = False)
org_yearly_pperson_v2_pivot.to_csv("org_yearly_tk_pperson_v2.csv", index = False)

In [ ]:
org_yearly_pay_pperson_v2

In [ ]:
pd.read_csv("s3://uci-capstone-custfields/org_yearly_tk_pperson_v2.csv")

In [ ]:
org_yearly_pperson_v2_pivot

In [ ]:
org_yearly_pperson_v2_pivot.to_csv("")

In [ ]:
org_yearly_pperson_v2_pivot.to_csv("org_yearly_tk_pperson_v2.csv")

# Close Connection

In [ ]:
cs.close()
omni.close()